## Rewrite of the "Baselines for mortality and LOS prediction" notebooks.
#### This notebook contains code to predict LOS using GRU-D, Random Forest and Logistic Regression in a simple notebook with explanations.

Author(s): Tomass Wilson, thwmi@kth.se

Credit: Shirly Wang, Matthew B. A. McDermott, Geeticka Chauhan, Michael C. Hughes, Tristan Naumann, 
and Marzyeh Ghassemi. MIMIC-Extract: A Data Extraction, Preprocessing, and Representation 
Pipeline for MIMIC-III. arXiv:1907.08322. 

### Setup

In [1]:
%load_ext autoreload

In [2]:
%autoreload

import copy, math, os, pickle, time, pandas as pd, numpy as np, scipy.stats as ss

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

import torch, torch.utils.data as utils, torch.nn as nn, torch.nn.functional as F, torch.optim as optim
from torch.autograd import Variable
from torch.nn.parameter import Parameter


from mmd_grud_utils import *

In [3]:
dirname = os.getcwd()
DATA_FILEPATH     = os.path.join(dirname, "..", "data", "all_hourly_data.h5")

In [4]:
GAP_TIME          = 6  # In hours
WINDOW_SIZE       = 24 # In hours
SEED              = 2
ID_COLS           = ['subject_id', 'hadm_id', 'icustay_id']
GPU               = '0'  # set this to the ID(s) of your most powerful GPU(s)

os.environ['CUDA_VISIBLE_DEVICES'] = GPU
np.random.seed(SEED)
torch.manual_seed(SEED)

### Load Data

In [5]:
%%time
data_full = pd.read_hdf(DATA_FILEPATH, 'vitals_labs')
statics        = pd.read_hdf(DATA_FILEPATH, 'patients')

Wall time: 12.6 s


### Create True/False mappings of the targets LOS 3/7 days
Also we create a list of subjects that all have lengths of stay that are at least 30 hours long.

Ys is a dataframe containing the target results for each subject (eg that they did stay for longer than 3 days).

data_df contains the metrics which we want to evaluate to predict length of stay. They are also the ones on which differential privacy will be applied.

In [6]:
# All patients in ys ahave been in icu at some point for at least 30 hours
Ys = statics[statics.max_hours > WINDOW_SIZE + GAP_TIME][['los_icu']]
Ys['los_3'] = Ys['los_icu'] > 3  # True/False column
Ys['los_7'] = Ys['los_icu'] > 7

# Get only the medical data of the chosen subjects
data_df = data_full[
    (data_full.index.get_level_values('icustay_id').isin(set(Ys.index.get_level_values('icustay_id')))) &
    (data_full.index.get_level_values('hours_in') < WINDOW_SIZE)
]

# Collect the subject id's
subj_idx, Ys_subj_idx = [df.index.get_level_values('subject_id') for df in (data_df, Ys)]
subjects = set(subj_idx)
assert subjects == set(Ys_subj_idx), "Subject ID pools differ!"

### Create train, dev and test fractions of the data

In [7]:
train_frac, dev_frac, test_frac = 0.7, 0.1, 0.2
np.random.seed(SEED)
subjects, N = np.random.permutation(list(subjects)), len(subjects)
N_train, N_dev, N_test = int(train_frac * N), int(dev_frac * N), int(test_frac * N)
train_subj = subjects[:N_train]
dev_subj   = subjects[N_train:N_train + N_dev]
test_subj  = subjects[N_train+N_dev:]

# Create train, dev and test fractions for data and Ys
[(data_train, data_dev, data_test), (Ys_train, Ys_dev, Ys_test)] = [
    [df[df.index.get_level_values('subject_id').isin(s)].copy() for s in (train_subj, dev_subj, test_subj)] \
    for df in (data_df, Ys)
]

### smthng

In [8]:
idx = pd.IndexSlice
data_means = data_train.loc[:, idx[:,'mean']].mean(axis=0)
data_stds = data_train.loc[:, idx[:,'mean']].std(axis=0)

data_train.loc[:, idx[:,'mean']] = (data_train.loc[:, idx[:,'mean']] - data_means)/data_stds
data_dev.loc[:, idx[:,'mean']] = (data_dev.loc[:, idx[:,'mean']] - data_means)/data_stds
data_test.loc[:, idx[:,'mean']] = (data_test.loc[:, idx[:,'mean']] - data_means)/data_stds

### smthng else

In [9]:
def simple_imputer(df):
    idx = pd.IndexSlice
    if len(df.columns.names) > 2: 
        df.columns = df.columns.droplevel(('label', 'LEVEL1', 'LEVEL2'))
    
    df_out = df.loc[:, idx[:, ['mean', 'count']]].copy()
    icustay_means = df_out.loc[:, idx[:, 'mean']].groupby(ID_COLS).mean()
    
    df_out.loc[:,idx[:,'mean']] = df_out.loc[:,idx[:,'mean']].groupby(ID_COLS).fillna(
        method='ffill'
    ).groupby(ID_COLS).fillna(icustay_means).fillna(0)
    
    df_out.loc[:, idx[:, 'count']] = (df.loc[:, idx[:, 'count']] > 0).astype(float)
    df_out.rename(columns={'count': 'mask'}, level='Aggregation Function', inplace=True)
    
    is_absent = (1 - df_out.loc[:, idx[:, 'mask']])
    hours_of_absence = is_absent.cumsum()
    time_since_measured = hours_of_absence - hours_of_absence[is_absent==0].fillna(method='ffill')
    time_since_measured.rename(columns={'mask': 'time_since_measured'}, level='Aggregation Function', inplace=True)

    df_out = pd.concat((df_out, time_since_measured), axis=1)
    df_out.loc[:, idx[:, 'time_since_measured']] = df_out.loc[:, idx[:, 'time_since_measured']].fillna(100)
    
    df_out.sort_index(axis=1, inplace=True)
    return df_out

data_train, data_dev, data_test = [
    simple_imputer(df) for df in (data_train, data_dev, data_test)
]

data_flat_train, data_flat_dev, data_flat_test = [
    df.pivot_table(index=['subject_id', 'hadm_id', 'icustay_id'], columns=['hours_in']) for df in (
        data_train, data_dev, data_test
    )
]

for df in data_train, data_dev, data_test: 
    assert not df.isnull().any().any()

## Prediction

### Hyperparameters

In [10]:
class DictDist():
    def __init__(self, dict_of_rvs): self.dict_of_rvs = dict_of_rvs
    def rvs(self, n):
        a = {k: v.rvs(n) for k, v in self.dict_of_rvs.items()}
        out = []
        for i in range(n): out.append({k: vs[i] for k, vs in a.items()})
        return out

class Choice():
    def __init__(self, options): self.options = options
    def rvs(self, n): return [self.options[i] for i in ss.randint(0, len(self.options)).rvs(n)]

In [23]:
N = 15

LR_dist = DictDist({
    'C': Choice(np.geomspace(1e-3, 1e3, 10000)),
    'penalty': Choice(['l1', 'l2']),
    'solver': Choice(['liblinear', 'lbfgs']),
    'max_iter': Choice([100, 500])
})
np.random.seed(SEED)
LR_hyperparams_list = LR_dist.rvs(N)
for i in range(N):
    if LR_hyperparams_list[i]['solver'] == 'lbfgs': LR_hyperparams_list[i]['penalty'] = 'l2'

RF_dist = DictDist({
    'n_estimators': ss.randint(50, 500),
    'max_depth': ss.randint(2, 10),
    'min_samples_split': ss.randint(2, 75),
    'min_samples_leaf': ss.randint(1, 50),
})
np.random.seed(SEED)
RF_hyperparams_list = RF_dist.rvs(N)

GRU_D_dist = DictDist({
    'cell_size': ss.randint(50, 75),
    'hidden_size': ss.randint(65, 95),
    'learning_rate': ss.uniform(2e-3, 1e-1),
    'num_epochs': ss.randint(60, 150),
    'patience': ss.randint(5, 9),
    'batch_size': Choice([64, 128, 256, 512]),
    'early_stop_frac': ss.uniform(0.05, 0.1),
    'seed': ss.randint(1, 10000),
})
np.random.seed(SEED)
GRU_D_hyperparams_list = GRU_D_dist.rvs(N)

In [12]:
results = {}  # Declare results dictionary

### Random Forest and Logistic Regression

In [17]:
def run_basic(model, hyperparams_list, X_flat_train, X_flat_dev, X_flat_test, target):
    best_s, best_hyperparams = -np.Inf, None
    for i, hyperparams in enumerate(hyperparams_list):
        print("On sample %d / %d (hyperparams = %s)" % (i+1, len(hyperparams_list), repr((hyperparams))))
        M = model(**hyperparams)
        M.fit(X_flat_train, Ys_train[target])
        s = roc_auc_score(Ys_dev[target], M.predict_proba(X_flat_dev)[:, 1])
        if s > best_s:
            best_s, best_hyperparams = s, hyperparams
            print("New Best Score: %.2f @ hyperparams = %s" % (100*best_s, repr((best_hyperparams))))

    return run_only_final(model, best_hyperparams, X_flat_train, X_flat_dev, X_flat_test, target)

def run_only_final(model, best_hyperparams, X_flat_train, X_flat_dev, X_flat_test, target):
    best_M = model(**best_hyperparams)
    best_M.fit(pd.concat((X_flat_train, X_flat_dev)), pd.concat((Ys_train, Ys_dev))[target])
    y_true  = Ys_test[target]
    y_score = best_M.predict_proba(X_flat_test)[:, 1]
    y_pred  = best_M.predict(X_flat_test)

    auc   = roc_auc_score(y_true, y_score)
    auprc = average_precision_score(y_true, y_score)
    acc   = accuracy_score(y_true, y_pred)
    F1    = f1_score(y_true, y_pred)
    
    return best_M, best_hyperparams, auc, auprc, acc, F1

In [18]:
for model_name, model, hyperparams_list in [
    ('RF', RandomForestClassifier, RF_hyperparams_list), ('LR', LogisticRegression, LR_hyperparams_list)
]:
    if model_name not in results: 
        results[model_name] = {}
    for t in ['los_7', 'los_3']:
        if t not in results[model_name]: 
            results[model_name][t] = {}
        
        if "data" in results[model_name][t]:
            print("Finished model %s on target %s with representation %s" % (model_name, t, "data"))
            if RERUN: 
                h = results[model_name][t]["data"][1]
                results[model_name][t]["data"] = run_only_final(model, h, data_flat_train, data_flat_dev, data_flat_test, t)

                print("Final results for model %s on target %s with representation %s" % (model_name, t, "data"))
                print(results[model_name][t]["data"][2:])

                with open(RESULTS_PATH, mode='wb') as f: pickle.dump(results, f)
            continue

        print("Running model %s on target %s with representation %s" % (model_name, t, "data"))
        results[model_name][t]["data"] = run_basic(
            model, hyperparams_list, data_flat_train, data_flat_dev, data_flat_test, t
        )
        print("Final results for model %s on target %s with representation %s" % (model_name, t, "data"))
        print(results[model_name][t]["data"][2:])  # auc, auprc, acc, F1
        # with open(RESULTS_PATH, mode='wb') as f: pickle.dump(results, f)

Running model RF on target los_7 with representation data
On sample 1 / 15 (hyperparams = {'n_estimators': 87, 'max_depth': 3, 'min_samples_split': 15, 'min_samples_leaf': 8})
New Best Score: 76.67 @ hyperparams = {'n_estimators': 87, 'max_depth': 3, 'min_samples_split': 15, 'min_samples_leaf': 8}
On sample 2 / 15 (hyperparams = {'n_estimators': 285, 'max_depth': 4, 'min_samples_split': 11, 'min_samples_leaf': 4})
New Best Score: 77.08 @ hyperparams = {'n_estimators': 285, 'max_depth': 4, 'min_samples_split': 11, 'min_samples_leaf': 4}
On sample 3 / 15 (hyperparams = {'n_estimators': 446, 'max_depth': 6, 'min_samples_split': 9, 'min_samples_leaf': 7})
New Best Score: 77.60 @ hyperparams = {'n_estimators': 446, 'max_depth': 6, 'min_samples_split': 9, 'min_samples_leaf': 7}
On sample 4 / 15 (hyperparams = {'n_estimators': 122, 'max_depth': 8, 'min_samples_split': 65, 'min_samples_leaf': 22})
On sample 5 / 15 (hyperparams = {'n_estimators': 305, 'max_depth': 7, 'min_samples_split': 63, 'm

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


On sample 3 / 15 (hyperparams = {'C': 0.003491839757169992, 'penalty': 'l2', 'solver': 'lbfgs', 'max_iter': 500})


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


On sample 4 / 15 (hyperparams = {'C': 48.783036208459954, 'penalty': 'l2', 'solver': 'lbfgs', 'max_iter': 100})


...

On sample 10 / 15 (hyperparams = {'C': 0.11951096159304532, 'penalty': 'l2', 'solver': 'lbfgs', 'max_iter': 100})


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


On sample 11 / 15 (hyperparams = {'C': 433.47464177487655, 'penalty': 'l2', 'solver': 'liblinear', 'max_iter': 500})
New Best Score: 67.25 @ hyperparams = {'C': 433.47464177487655, 'penalty': 'l2', 'solver': 'liblinear', 'max_iter': 500}
On sample 12 / 15 (hyperparams = {'C': 1.7295128471299193, 'penalty': 'l2', 'solver': 'liblinear', 'max_iter': 500})
On sample 13 / 15 (hyperparams = {'C': 1.6777315525707752, 'penalty': 'l2', 'solver': 'lbfgs', 'max_iter': 500})


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


On sample 14 / 15 (hyperparams = {'C': 0.059893230394471704, 'penalty': 'l1', 'solver': 'liblinear', 'max_iter': 500})
New Best Score: 68.87 @ hyperparams = {'C': 0.059893230394471704, 'penalty': 'l1', 'solver': 'liblinear', 'max_iter': 500}
On sample 15 / 15 (hyperparams = {'C': 0.032340816118081595, 'penalty': 'l1', 'solver': 'liblinear', 'max_iter': 100})
New Best Score: 69.22 @ hyperparams = {'C': 0.032340816118081595, 'penalty': 'l1', 'solver': 'liblinear', 'max_iter': 100}
Final results for model LR on target los_3 with representation data
(0.6999474339003102, 0.6382947152889347, 0.6680584551148225, 0.5558659217877094)


### GRU-D

In [80]:
%autoreload
from mmd_grud_utils import *

model_name       = 'GRU-D'
hyperparams_list = GRU_D_hyperparams_list
RERUN            = False
results = {}

if model_name not in results: 
    results[model_name] = {}

for t in ['los_3', 'los_7']:
    if t not in results[model_name]: 
        results[model_name][t] = {}
    n, X_train, X_dev, X_test = ('data', data_train, data_dev, data_test)
    
    print("Running model %s on target %s with representation %s" % (model_name, t, n))
    tensor = to_3D_tensor(
            X_train.loc[:, pd.IndexSlice[:, 'mean']] * 
            np.where((X_train.loc[:, pd.IndexSlice[:, 'mask']] == 1).values, 1, np.NaN)
        )
    X_mean = np.nanmean(tensor, axis=0, keepdims=True).transpose([0, 2, 1])
    X_mean[X_mean != X_mean] = 0  # THIS IS EXTREME BODGE. We should not set NaN values to 0, as that wrongly influences training
    base_params = {'X_mean': X_mean, 'output_last': True, 'input_size': X_mean.shape[2]}

    if n in results[model_name][t]:
        if not RERUN: 
            print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
            print(results[model_name][t][n])
            continue
        best_s, best_hyperparams = results[model_name][t][n][-1], results[model_name][t][n][1]
        print("Loading best hyperparams", best_hyperparams)
    
    else:
        best_s, best_hyperparams = -np.Inf, None
        for i, hyperparams in enumerate(hyperparams_list):
            print("On sample %d / %d (hyperparams = %s)" % (i+1, len(hyperparams_list), repr((hyperparams))))

            early_stop_frac,batch_size,seed = [hyperparams[k] for k in ('early_stop_frac','batch_size','seed')]

            batch_size = int(batch_size)

            np.random.seed(seed)
            all_train_subjects = list(
                np.random.permutation(Ys_train.index.get_level_values('subject_id').values)
            )
            N_early_stop        = int(len(all_train_subjects) * early_stop_frac)
            train_subjects      = all_train_subjects[:-N_early_stop]
            early_stop_subjects = all_train_subjects[-N_early_stop:]
            X_train_obs         = X_train[X_train.index.get_level_values('subject_id').isin(train_subjects)]
            Ys_train_obs        = Ys_train[Ys_train.index.get_level_values('subject_id').isin(train_subjects)]

            X_train_early_stop  = X_train[X_train.index.get_level_values('subject_id').isin(early_stop_subjects)]
            Ys_train_early_stop = Ys_train[
                Ys_train.index.get_level_values('subject_id').isin(early_stop_subjects)
            ]

            train_dataloader      = prepare_dataloader(X_train_obs, Ys_train_obs[t], batch_size=batch_size)
            early_stop_dataloader = prepare_dataloader(
                X_train_early_stop, Ys_train_early_stop[t], batch_size=batch_size
            )
            dev_dataloader        = prepare_dataloader(X_dev, Ys_dev[t], batch_size=batch_size)
            test_dataloader       = prepare_dataloader(X_test, Ys_test[t], batch_size=batch_size)

            model_hyperparams = copy.copy(base_params)
            model_hyperparams.update(
                {k: v for k, v in hyperparams.items() if k in ('cell_size', 'hidden_size', 'batch_size')}
            )
            model = GRUD(**model_hyperparams)

            best_model, _ = Train_Model(
                model, train_dataloader, early_stop_dataloader,
                **{k: v for k, v in hyperparams.items() if k in (
                    'num_epochs', 'patience', 'learning_rate', 'batch_size'
                )}
            )

            probabilities_dev, labels_dev = predict_proba(best_model, dev_dataloader)
            probabilities_dev = np.concatenate(probabilities_dev)[:, 1]
            labels_dev        = np.concatenate(labels_dev)
            s = roc_auc_score(labels_dev, probabilities_dev)
            if s > best_s:
                best_s, best_hyperparams = s, hyperparams
                print("New Best Score: %.2f @ hyperparams = %s" % (100*best_s, repr((best_hyperparams))))
                
    ## Test set
    np.random.seed(seed)
    early_stop_frac,batch_size,seed = [best_hyperparams[k] for k in ('early_stop_frac','batch_size','seed')]
    
    batch_size = int(batch_size)

    X_train_concat, Ys_train_concat = pd.concat((X_train, X_dev)), pd.concat((Ys_train, Ys_dev))

    all_train_subjects = list(np.random.permutation(Ys_train_concat.index.get_level_values('subject_id').values))
    N_early_stop = int(len(all_train_subjects) * early_stop_frac)
    train_subjects, early_stop_subjects = all_train_subjects[:-N_early_stop], all_train_subjects[-N_early_stop:]
    X_train_obs         = X_train_concat[X_train_concat.index.get_level_values('subject_id').isin(train_subjects)]
    Ys_train_obs        = Ys_train_concat[Ys_train_concat.index.get_level_values('subject_id').isin(train_subjects)]

    X_train_early_stop  = X_train_concat[X_train_concat.index.get_level_values('subject_id').isin(early_stop_subjects)]
    Ys_train_early_stop = Ys_train_concat[Ys_train_concat.index.get_level_values('subject_id').isin(early_stop_subjects)]

    train_dataloader      = prepare_dataloader(X_train_obs, Ys_train_obs[t], batch_size=batch_size)
    early_stop_dataloader = prepare_dataloader(X_train_early_stop, Ys_train_early_stop[t], batch_size=batch_size)
    test_dataloader       = prepare_dataloader(X_test, Ys_test[t], batch_size=batch_size)

    model_hyperparams = copy.copy(base_params)
    model_hyperparams.update(
        {k: v for k, v in best_hyperparams.items() if k in ('cell_size', 'hidden_size', 'batch_size')}
    )
    model = GRUD(**model_hyperparams)

    best_model, (losses_train, losses_early_stop, losses_epochs_train, losses_epochs_early_stop) = Train_Model(
        model, train_dataloader, early_stop_dataloader,
        **{k: v for k, v in best_hyperparams.items() if k in (
            'num_epochs', 'patience', 'learning_rate', 'batch_size'
        )}
    )

    probabilities_test, labels_test = predict_proba(best_model, test_dataloader)

    y_score = np.concatenate(probabilities_test)[:, 1]
    y_pred  = np.concatenate(probabilities_test).argmax(axis=1)
    y_true  = np.concatenate(labels_test)

    auc   = roc_auc_score(y_true, y_score)
    auprc = average_precision_score(y_true, y_score)
    acc   = accuracy_score(y_true, y_pred)
    F1    = f1_score(y_true, y_pred)
    print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
    print(auc, auprc, acc, F1)

    results[model_name][t][n] = None, best_hyperparams, auc, auprc, acc, F1, best_s
    # with open('/scratch/mmd/extraction_baselines_gru-d.pkl', mode='wb') as f: pickle.dump(results, f)

Running model GRU-D on target los_3 with representation data


<ipython-input-80-7d35c2d9607d>:22: RuntimeWarning: Mean of empty slice
  X_mean = np.nanmean(tensor, axis=0, keepdims=True).transpose([0, 2, 1])


On sample 1 / 15 (hyperparams = {'cell_size': 58, 'hidden_size': 91, 'learning_rate': 0.056620731990215806, 'num_epochs': 30, 'patience': 5, 'batch_size': 256, 'early_stop_frac': 0.09260225260560727, 'seed': 5121})
Model Structure:  GRUD(
  (zl): Linear(in_features=299, out_features=91, bias=True)
  (rl): Linear(in_features=299, out_features=91, bias=True)
  (hl): Linear(in_features=299, out_features=91, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=91, bias=True)
  (fc): Linear(in_features=91, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.82396394, valid_loss: 0.80602096, time: [5.03], best model: 1
Epoch: 1, train_loss: 0.72999637, valid_loss: 0.74542825, time: [5.03], best model: 1
Epoch: 2, train_loss: 0.712

<ipython-input-80-7d35c2d9607d>:22: RuntimeWarning: Mean of empty slice
  X_mean = np.nanmean(tensor, axis=0, keepdims=True).transpose([0, 2, 1])


On sample 1 / 15 (hyperparams = {'cell_size': 58, 'hidden_size': 91, 'learning_rate': 0.056620731990215806, 'num_epochs': 30, 'patience': 5, 'batch_size': 256, 'early_stop_frac': 0.09260225260560727, 'seed': 5121})
Model Structure:  GRUD(
  (zl): Linear(in_features=299, out_features=91, bias=True)
  (rl): Linear(in_features=299, out_features=91, bias=True)
  (hl): Linear(in_features=299, out_features=91, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=91, bias=True)
  (fc): Linear(in_features=91, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.84016884, valid_loss: 0.85484075, time: [5.12], best model: 1
Epoch: 1, train_loss: 0.75291766, valid_loss: 0.76328109, time: [5.04], best model: 1
Epoch: 2, train_loss: 0.686